# Import Libraries

In [87]:
import glob

import pandas as pd
from pytrends.request import TrendReq
from pytrends import dailydata
from datetime import date


# Import CSVs

In [88]:
df = pd.read_csv("GoogleTrends.csv")

# Using Trends

In [89]:
# Timezone Offset(US CST = 360)
pytrends = TrendReq(hl='en-US', tz=360)
weekly_dates = pd.date_range(start='2017-01-01', end='2020-12-31', freq='W')

# build the payload
def calculate_trend(key):
    pytrends.build_payload([key], cat=0, timeframe='2017-01-01 2020-12-31', geo='US')
    weekly_trends = pytrends.interest_over_time()
    return weekly_trends

weekly_trend_df = pd.DataFrame(index=pd.date_range(date(2017,1,1), date(2020,12,31), freq='W'))
adjusted_df = pd.DataFrame(index=pd.date_range(date(2017,1,1), date(2020,12,31), freq='D'))

for column in df:
    if not "Unnamed" in df[column].name:
        print(df[column].name)
        adjusted_df[df[column].name] = 0
        weekly_trends = calculate_trend(df[column].name)
        weekly_trend_df[df[column].name] = weekly_trends.iloc[:, 0]



for index, row in df.iterrows():
    row_date = row.iloc[0]
    weekly_trends_row = (weekly_trend_df.loc[weekly_trend_df.index <= row_date]).iloc[-1]
    weekly_trends_row_adj = weekly_trends_row.divide(100)
    new_row = row.iloc[1:].mul(weekly_trends_row_adj)
    adjusted_df.iloc[index] = new_row



adjusted_df.to_csv("GoogleTrendsAdjusted.csv", index=True)

pandemic
covid
covid-19
coronavirus
cases
cdc
fauci
recession
crisis
deaths
restrictions
lockdown
vaccine
vaccines
travel
stimulus
variant
pfizer
